In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import time
import pandas as pd
import pymongo as mg

In [2]:
# MongoDB 연결
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_nutrients']
collection = database['11st_corelsterolsupplement_images']

In [3]:
# Chrome 웹 드라이버 설정
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [4]:
# open chrome browser
browser = webdriver.Chrome(executable_path='C:/Users/04-02/Develops/chromedriver.exe', options=options)
browser.set_window_size(1560,2000)

In [5]:
# url in address window 리뷰 많은 순
browser.get('https://search.11st.co.kr/pc/total-search?kwd=%25EC%25BD%259C%25EB%25A0%2588%25EC%258A%25A4%25ED%2585%258C%25EB%25A1%25A4%2520%25EC%2598%2581%25EC%2596%2591%25EC%25A0%259C&fromACK=recent&tabId=TOTAL_SEARCH&sortCd=I&pageNo=1')
browser.implicitly_wait(10)

In [6]:
# pagination 하기
product_total = browser.find_element_by_css_selector('div.search_content > div > p > span')
product_total_str = product_total.text
product_total_del = product_total_str.replace(',', '')
product_total_count = int(product_total_del)

In [7]:
print(product_total_count) 

# 한 페이지 상품 목록 수
product_standard_count_per = 60 

6288


In [8]:
# 상품 총 페이지수
loop_count_int = int(product_total_count / product_standard_count_per) 
print(loop_count_int)  

104


In [9]:
## 혜인설명: 총 댓글 수를 정규화로 뽑아냄 .
# 정규 표현식을 사용하여 숫자 추출 (',' 포함)
import re

In [10]:
# 상품 이름 (f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > div:nth-child(3) > div.c-card-item__info-top > dl > div.c-card-item__name > dd').text 
# 상품 링크 (f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > a').get_attribute('href')
# 상품 이미지 (f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > div.c-card-item__thumb > span > img').get_attribute('src')
# 상품 댓글 총 갯수 (f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > div:nth-child(3) > div.c-card-item__info-top > dl > div.c-starrate > dl > dd:nth-child(3) > span > span.value').text

In [11]:
current_page = 1

# 이미지 및 리뷰 데이터를 저장할 리스트 초기화
images_links_list = []

# pagination 버튼을 끝까지 순환하면서 페이지 이동합니다.
while current_page <= loop_count_int:
    for i in range(1, 61):  # 1부터 60까지 순회합니다.
        try:
            product_name = browser.find_element_by_css_selector(f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > div:nth-child(3) > div.c-card-item__info-top > dl > div.c-card-item__name > dd').text   
            product_link = browser.find_element_by_css_selector(f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > a').get_attribute('href')
            product_image = browser.find_element_by_css_selector(f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > div.c-card-item__thumb > span > img').get_attribute('src')

            review_total_count_text = browser.find_element_by_css_selector(f'#section_commonPrd > div.c-search-list > ul > li:nth-child({i}) > div > div:nth-child(3) > div.c-card-item__info-top > dl > div.c-starrate > dl > dd:nth-child(3) > span > span.value').text

            # 정규 표현식을 사용하여 숫자 추출 (',' 포함)
            review_count = re.findall(r'\d+', review_total_count_text)
            
            # 추출된 숫자들을 하나의 문자열로 결합하고 ',' 제거
            review_count = ''.join(review_count)
            
            # 문자열을 정수로 변환
            review_total_count = int(review_count)  # 리뷰 총 갯수 

            # if review_total_count <= 10:
            #     print('리뷰가 10 이하인 상품이 발견되어 코드 종료합니다.')
            #     current_page = loop_count_int + 1  # 루프 종료 조건을 만족하도록 페이지 번호 업데이트
            #     break

            # 데이터를 리스트에 추가
            image_link_list = [product_name, product_link, product_image, review_total_count]
            images_links_list.append(image_link_list)

        except:
            print(f'에러: {i}')
            pass

    # 현재 페이지 번호를 업데이트
    current_page += 1

# MongoDB에 데이터 저장
if images_links_list:
    # 데이터프레임 생성
    review_columns_name = ["product_name", "product_link", "product_image", "review_total_count"]
    df_images = pd.DataFrame(data=images_links_list, columns=review_columns_name)

    # MongoDB에 데이터 저장
    data_dict = df_images.to_dict(orient='records')
    collection.insert_many(data_dict)

# Pagination 버튼을 클릭합니다.
# 다음 페이지 버튼이 클릭 가능할 때까지 대기 
# current_page += 1
# page_button_css = f'#section_commonPrd > nav > ul > li:nth-child({current_page % 10 + 2}) > button'
# page_button = browser.find_element_by_css_selector(page_button_css)
# page_button.click()

# # 10번째 페이지일 경우 next 버튼 클릭
# if current_page % 10 == 0:
#     next_button = browser.find_element_by_css_selector('#section_commonPrd > nav > ul > li.next > button')
#     next_button.click()

# 웹 드라이버 종료
browser.quit()